In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Purpose of this note book
The purpose of this notebook is to explore data provide in a very simple way, so that kaggle beginner (like me) can understand the overview of competition. This notebook includes contents bellow.

**Contents**<br>
- Reading input data
- Visualize & Analyze data
- Feautures of cassava desease
- Cassava classification quize to train your own nueral network!!

# Imports

Importing neccssary pakages for explore

In [ ]:
import glob
import json
import PIL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

# Reading data

We are going to read data bellow 
1. Json file to map desease name and label data
2. Label csv file
3. File path of train images

In [ ]:
# ID code and desease name mapping
with open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json") as f:
    desease_map = json.load(f)

In [ ]:
# Read label CSV 
train_label = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")


In [ ]:
# Get jpeg file path
train_image_files = glob.glob("../input/cassava-leaf-disease-classification/train_images/*.jpg")
print(len(train_image_files))

# Visualize & Analyze data

Here I am going to visualize data loaded at above cells. I explore label data first, and then image data.

## Explore label data

Let's take a look at label data first to understand the disribution of data.

In [ ]:
# Show head and tails in label data.
display(train_label.head())
display(train_label.tail())

Looks like there is 21396 data. <br>
Each row conists of "file name" and "label id". Let's find out what each label represents.

In [ ]:
# Here are the label (that is defined by competition) and its actual desease name.
display(desease_map)

lable=4 indicates "Healthy" and others are desease. I am going to show the features for each desease at "Feautures on cassava desease" part in this notebook.<br>

Next, let's get statisical overivew of label data.

In [ ]:
# Display summary info for label data
display(train_label.info())
display(train_label.describe())
print(train_label.isna().sum()) # Counting number of misiing values

From output above we can say...
1. No misiing value in label data
2. Max value for label is 4, which means undefined label is not included.
3. Maen value for label is 2.65 so that number of data is little iclined to higher value.<br>
   (We are going to see the distribution next for precise discussion)

Next I am going to see the distribution of labels. This inspection is very important because, biased data whitout reason lead to bad model.

In [ ]:
# See distribition of label data
count_per_label = train_label["label"].value_counts(sort=False)
ratio_per_label = train_label["label"].value_counts(sort=False, normalize=True)
desease_id = pd.DataFrame(desease_map.keys())
desease_name = pd.DataFrame(desease_map.values())

# Show distribution for label by tabel
print("Number of data per label ")
comb = pd.concat([desease_id, desease_name, count_per_label, ratio_per_label], axis=1)
comb.columns = ["Label id", "Desease name", 'Number counts', 'Ratio']
display(comb)

# Show distribution for label by bar graph and pie chart 
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))
# Plot Bar graph
comb.plot.bar(x="Label id", y="Number counts",ax=ax[0], title="Number of desease per id", rot=0)
# Plot Pie chart
comb.set_index("Desease name").plot.pie(y="Ratio", ax=ax[1], title="Ratio of desease",autopct='%1.1f%%')
plt.plot()

Bellow is the information that I got from above figures and numbers

**Summary for label data**<br>
- Key is image file name, Value is code to indentify disease.
- 21397 datas with no missing value.
- id=3('Cassava Mosaic Disease (CMD)') is the most common desease (60%), and number of the others are almost same (5-10%)
- ID code and desease name mapping is below <br>
  '0': 'Cassava Bacterial Blight (CBB)',<br>
  '1': 'Cassava Brown Streak Disease (CBSD)',<br>
  '2': 'Cassava Green Mottle (CGM)',<br>
  '3': 'Cassava Mosaic Disease (CMD)',<br>
  '4': 'Healthy'

## Images

### Image size

First I am going to see the size (width and height) of an image.

In [ ]:
# Image size distribution
image_size = []
for path in train_image_files:
    size = PIL.Image.open(path).size
    image_size.append(size)

In [ ]:
# Create DataFrame to for image size
img_size_df = pd.DataFrame(image_size, columns=["Width", "Hight"])

In [ ]:
# Plot image size distribution on 2D graph
img_size_df.plot(kind="scatter", x="Width", y="Hight")

In [ ]:
# Unique values in img_size_df
print(img_size_df.value_counts())

Looks like all the train images has the same size (width=800, height=600)! <br>
This means that we don't have to extract too big/small images before training.

## Show images

Finally I am going to show some image per each desease label.<br>
First I define functios.

In [ ]:
# Extract image file path with certain label
def pick_img_by_label(X, y, label, num=10):
    label_i = X[y==label]
    return label_i[np.random.choice(len(label_i), num, replace=False)]

In [ ]:
# Show image with certain label for image number specified. 
def show_image_per_label(X, y, label, num=9, figsize=(20,20)):
    X_label_i = pick_img_by_label(X, y, label, num)
    rows, cols = np.ceil(num/3), 3
    
    plt.figure(figsize=figsize)
    for j, file in enumerate(X_label_i):
        img = cv2.imread(file)
        plt.subplot(rows, cols, j+1)
        plt.axis("off")
        plt.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
train_label

In [ ]:
# Add file path column
train_label["file_path"] = "../input/cassava-leaf-disease-classification/train_images/" + train_label["image_id"]
# Create X, y data
X_files = train_label["file_path"].values
y = train_label["label"].values

# Show 10 images per label
for label in range(5):
    print("Label: {}, Desease name: {}".format(label, desease_map[str(label)]))
    show_image_per_label(X_files, y, label, num=9, figsize=(10,10))

# Feautures on cassava desease

**Breif servey on each desease**<br>

Bellow are the breif surevey summary for each desease.

Label 0: Cassava Bacterial Blight (CBB)
 - Symptoms include blight, wilting, dieback, and vascular necrosis.<br>
  (source:https://en.wikipedia.org/wiki/Bacterial_blight_of_cassava)

Label 1: Cassava Brown Streak Disease (CBSD)
 - CBSD is characterized by severe chlorosis and necrosis on infected leaves, giving them a yellowish, mottled appearance.
 - Symptons in leaf are sumillar to that of CMD, however CMD shows more apparent symptons.
 - CBSD is as serious as CMD in Africa, however CBSD is not found in Asia.<br>
 (sorce: <br>
  - https://en.wikipedia.org/wiki/Cassava_brown_streak_virus_disease
  - https://www.jica.go.jp/project/all_asia/005/materials/ku57pq000025s2lv-att/cassava_about.pdf)

Lable 2: Cassava Green Mottle (CGM)
 - Sympotons includes yellow spots, green mosiac patter and twisted margin.
 - 
(sorce: http://www.pestnet.org/fact_sheets/v6/cassava_green_mottle_068.htm)

Lable 3: Cassava Mosaic Disease (CMD)
 - CMD makes yellow or yellow-green spot on leaf.
 - Cassava Mosaic Virus is  virus that decreases crop yeild.<br>
 (source:https://en.wikipedia.org/wiki/Cassava_mosaic_virus)

**Disscussion**
- Data with label=3 (Cassava Mosaic Disease) has the most number might be because label=3 is a seirious desease for cassava in terms of yeild ammount.
- Looks difficult to classify label 1, 2 and 3 as there sympton are simillar.

# Cassava desease quize

This part is a quize for clasifying cassava desease by own nueral network! I hope this quize gives you an insight on features of each desease or images

In [ ]:
# Define function to run quize
def cassava_quize(X=X_files, y=y, quize_num=5, figsize=(10,10)):
    rand_idx = np.random.choice(len(X), quize_num, replace=False)
    ans = [str(y[img_idx]) for img_idx in rand_idx]

    nrows, ncols = int(np.ceil(quize_num / 3)), 3
    plt.figure(figsize=figsize)
    for i, img_idx in enumerate(rand_idx):
        img = cv2.imread(X[img_idx])
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title("Image {}".format(i+1))
    plt.tight_layout()
    plt.show()

    return ans

def your_answer(ans):
    my_ans = []
    print("Answer selections")
    for key, val in desease_map.items():
        print("Label:{}, Name:{}".format(key, val))
    for j in range(len(ans)):
        ans_j = input("Type your answer for Image {} ".format(j+1))
        my_ans.append(ans_j)
    print("Your answer: {}".format(my_ans))
    print("Correct answer: {}".format(ans))
    print("Results: {}/{}".format(str(sum(np.array(my_ans)==np.array(ans))), str(len(ans))))

**How to play quize**
1. Run first cell to show problem
2. Run second cell to type your answer
3. Look at the result and get insight for cassava desease or images!

In [ ]:
# 1st Step: Run here to show problem
ans = cassava_quize()

In [ ]:
# 2nd-3rd Step: Run here to type your answer and check result!!
your_answer(ans)